In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import zipfile
import re
import dateparser
from lxml import etree
from xml.etree.ElementTree import XML
docxFileName = "quick_test.docx"
docxZip = zipfile.ZipFile(docxFileName)
documentXML = docxZip.read('word/document.xml')
et = etree.XML(documentXML)
ns = {'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}

In [2]:
mainstation_xpath = './w:r[1][not(w:rPr/w:sz[@w:val="16"])]/w:t[1]'
lowerstation_xpath = './w:r[1][not(w:rPr/w:sz[@w:val="18"])]/w:t[1]'
first_token_para_xpath = './w:r[1]/w:t[1]'

stations = pd.DataFrame(columns=['station','type','description'])

def station_type (para):
    #if para.xpath(first_token_para_xpath, namespaces=ns)[0].text.islower() == 'True':
        #return 'na'
    if para.xpath(mainstation_xpath, namespaces=ns):
        return 'main'        
    elif para.xpath(lowerstation_xpath, namespaces=ns):
        return 'lower'
    else:
        return 'na'


def is_upper(para):
    if para.xpath(first_token_para_xpath, namespaces=ns):
        if para.xpath(first_token_para_xpath, namespaces=ns)[0].text.isupper():
            return 'station'
        else: 
            return 'new_line'


for i, para in enumerate(et.xpath('//w:p', namespaces=ns)):
    text = para.xpath('./w:r/w:t', namespaces=ns)
    description = " ".join([t.text for t in text]) 
    if is_upper(para) == 'station':
        if station_type(para) =='main':
            mainstation = para.xpath(mainstation_xpath, namespaces=ns)            
            stations.at[i,'station'] = mainstation[0].text
            stations.at[i,'type'] = 'main'
            #print("MAIN:", mainstation[0].text)
        elif station_type(para) =='lower':
            lowerstation = para.xpath(lowerstation_xpath, namespaces=ns)
            stations.at[i,'station'] = lowerstation[0].text
            stations.at[i,'type'] = 'secondary'
            #print("SUB:", lowerstation[0].text)
    else:
        stations.at[i,'station'] = '-'
        stations.at[i,'type'] = '-'
    stations.at[i,'description'] = description
    #print("DESCRIPTION:", description)
    #print()


stations.head(3)
pd.options.display.max_colwidth = 100
#stations.to_csv('output.csv')


##notes: 
#"ABERDARE JUNCTION  see ABERCYNON." ("JUNCTION" in document.xml isn't marked-up correctly)
#"Bridge over line near Schoolhill ticket platform (ibid 8 June 1895). Separate structure or station used as collection point for..." ("Bridge" in document.xml isn't marked-up correctly)
    
    


In [3]:
stations.head(5)

,station,type,description
0,ABBEY,main,ABBEY
1,A TOWN,secondary,A TOWN [NB] op 3 September 1856** as A; TOWN added 1889 tt (Cl) ; clo 7 September 1964 (RM O...
2,A JUNCTION,secondary,A JUNCTION [Cal] op 31 August 1870 (co ½ T 26 September) . Clo 20 May 1921 (see 192 1 April/...
3,A JUNCTION,secondary,A JUNCTION [NB] op 8 August 1870 (D&C 14) ; clo 1 September 1921 (Thursday) (RCG) – stops s...
4,-,-,Brad and co tts seen all have Abbey Junction for the station concerned. OS maps show as effecti...


In [26]:
def find_between(s, start, end):
  return s.partition(start)[2].partition(end)[0]

def railway_company(s):
  pattern = re.compile('\[[a-zA-Z]+\]')
  m = pattern.findall(s)
  if m:
    return m[0].partition('[')[2].partition(']')[0]



def date_open(s):
  pattern = re.compile(r'op\s.*?\bclo\b')
  m = pattern.findall(s)
  if m:
     return m[0].partition('op')[2].partition('clo')[0].strip()
  else:
    pattern = re.compile(r'op\s.*?[0-9]{4}')
    m = pattern.findall(s)
    if m:
      return m[0].partition('op')[2].strip()


def date_closed(s):
  pattern = re.compile(r'clo\s.*?[.]')
  m = pattern.findall(s)
  if m:
     return m[0].partition('clo')[2].strip()
  else:
      pattern = re.compile(r'still open')
      m = pattern.findall(s)
      if m:
        return m[0].strip()


def place_disambiguation(s):
  pattern = re.compile(r'[A-Z]+\s*near\s*[a-zA-Z]*\s')
  m = pattern.findall(s)
  if m:
     return m[0].partition('near')[2].strip()


def additional_notes(s):
  end = date_closed(s)
  if end:
    notes = s.partition(str(end))[2]
    if notes:
      return notes.strip()

stations['place_disambiguation'] = stations.description.apply(lambda x: place_disambiguation(x))
stations['railway_company'] = stations.description.apply(lambda x: railway_company(x))
stations['open'] = stations.description.apply(lambda x: date_open(x.lower()))
stations['closed'] = stations.description.apply(lambda x: date_closed(x.lower()))
stations['additional_notes'] = stations.description.apply(lambda x: additional_notes(x))
stations

stations.to_csv('output.csv', encoding='utf-8')


In [20]:
#####TEST SANDBOX####

s =  "ABER near Caerphilly [Rhy] op 1 April 1908 (Aber) as BEDDAU (BEDA, error, in Brad); became A JUNCTION 17 September 1926 (GW circ 3011), A 6 May 1968 (Aberdare); HALT "
pattern = re.compile(r'[A-Z]+\s*near\s*[a-zA-Z]*\s')
m = pattern.findall(s)
if m:
    print(m[0])

ABER near Caerphilly 
